Installing the dependencies

In [11]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install localtunnel  # Install LocalTunnel
!pip install pyngrok
!pip install nest-asyncio

  Using cached pickle5-0.0.11.tar.gz (132 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pickle5


  error: subprocess-exited-with-error
  
  × Building wheel for pickle5 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-38\pickle5
      copying pickle5\pickle.py -> build\lib.win-amd64-cpython-38\pickle5
      copying pickle5\pickletools.py -> build\lib.win-amd64-cpython-38\pickle5
      copying pickle5\__init__.py -> build\lib.win-amd64-cpython-38\pickle5
      creating build\lib.win-amd64-cpython-38\pickle5\test
      copying pickle5\test\pickletester.py -> build\lib.win-amd64-cpython-38\pickle5\test
      copying pickle5\test\test_pickle.py -> build\lib.win-amd64-cpython-38\pickle5\test
      copying pickle5\test\test_picklebuffer.py -> build\lib.win-amd64-cpython-38\pickle5\test
      copying pickle5\test\__init__.py -> build\lib.win-amd64-cpython-38\pickle5\test
      running build_ext
      building 'pickle5._pickle' extension
   

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [12]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
import subprocess
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [13]:
app = FastAPI()

In [14]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [15]:
class model_input(BaseModel):
    
    Pregnancies : int
    Glucose : int
    BloodPressure : int
    SkinThickness : int
    Insulin : int
    BMI : float
    DiabetesPedigreeFunction : float
    Age : int

In [16]:
# loading the saved model
loaded_model = pickle.load(open('D:/ML_with_Python/Deployment_ML/trained_model.sav', 'rb'))

In [17]:
@app.post('/diabetes_prediction')
def diabetes_predd(input_parameters : model_input):
    
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)
    
    preg = input_dictionary['Pregnancies']
    glu = input_dictionary['Glucose']
    bp = input_dictionary['BloodPressure']
    skin = input_dictionary['SkinThickness']
    insulin = input_dictionary['Insulin']
    bmi = input_dictionary['BMI']
    dpf = input_dictionary['DiabetesPedigreeFunction']
    age = input_dictionary['Age']
    
    
    input_list = [preg, glu, bp, skin, insulin, bmi, dpf, age]
    
    prediction = loaded_model.predict([input_list])
    
    if prediction[0] == 0:
        return 'The person is not diabetic'
    else:
        return 'The person is diabetic'

In [20]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Create a tunnel to localhost:8000
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

# Resolve asyncio issues with Jupyter Notebook
nest_asyncio.apply()

# Start the app
uvicorn.run(app, port=8000)

INFO:     Started server process [11572]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://ebe6-2402-8100-27d0-4db7-f179-3775-bc99-a92.ngrok-free.app
INFO:     2402:8100:27d0:4db7:f179:3775:bc99:a92:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2402:8100:27d0:4db7:f179:3775:bc99:a92:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2402:8100:27d0:4db7:f179:3775:bc99:a92:0 - "POST /diabetes_prediction HTTP/1.1" 200 OK


C:\Users\sachi\AppData\Local\Temp\ipykernel_11572\99619241.py:4: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  input_data = input_parameters.json()
c:\Users\sachi\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [11572]
